In [1]:
import numpy as np
from numpy import asarray
from numpy import arange
from numpy.random import rand
from numpy.random import seed
import matplotlib.pyplot as plt
import seaborn as sns
from func_autograd import *
from sklearn.model_selection import train_test_split
import pandas as pd

def MSE(y_data, y_model):
	n = np.size(y_model)
	y_data = y_data.reshape(-1,1)
	y_model = y_model.reshape(-1,1)
	return np.sum((y_data - y_model)**2)/n


def generate_data(noise=True, step_size=0.05 , FrankesFunction=True):
    # Arrange x and y
    x = np.arange(0, 1, step_size)
    y = np.arange(0, 1, step_size)
    # Create meshgrid of x and y
    X, Y = np.meshgrid(x, y)
    
    if FrankesFunction:
        # Calculate the values for Franke function
        z = FrankeFunction(X, Y, noise=noise).flatten()
    else:
        z = TestFunction(X, Y, noise=noise).flatten()

    # Flatten x and y for plotting
    x = X.flatten()
    y = Y.flatten()
    
    return x, y, z

def TestFunction(x, y, noise=False):
    if noise: 
        random_noise = np.random.normal(0, 0.1 , x.shape)
    else: 
        random_noise = 0

    return  x**2 + y**2 + 2*x*y + random_noise

def FrankeFunction(x, y, noise=False):
    if noise: 
        random_noise = np.random.normal(0, 0.1 , x.shape)
    else: 
        random_noise = 0
    
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4 + random_noise

x, y, z = generate_data()
X = create_X(x, y, 7)
X_train, X_test, z_train, z_test = train_test_split(X, z)


# parameters
gamma = np.linspace(0.001, 0.1, 10)
delta = np.linspace(0.05, 0.5, 10)
eta = np.linspace(0.7, 0.95, 5)

# OLS
beta = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ z_train
MSE_OLS_train = MSE(X_train @ beta, z_train)
MSE_OLS_test = MSE(X_test @ beta, z_test)

X_train = X_train[:, 1:3]
X_test = X_test[:, 1:3]

trials = 10

gd_gamma = np.zeros(len(gamma))
gd_mom_gamma = np.zeros(len(gamma))
gd_mom_delta = np.zeros(len(delta))
sgd_mom_gamma = np.zeros(len(gamma))
sgd_mom_delta = np.zeros(len(delta))
sgd_mom_eta = np.zeros(len(eta))

for t in range(trials):
    initial_val = np.random.randn(X_train.shape[1],1)

    # plain gradient descent with fixed learning rate using analytic expression of gradient
    train_score = np.zeros(len(gamma))
    for i in range(len(gamma)):
        model = GradientDescend(momentum=False, learning_rate=gamma[i])
        scores = model.fit(X_train, z_train, X_test, z_test)

        pred_train = model.predict(X_train)
        train_score[i] = MSE(pred_train, z_train)

    i_min, min = train_score.argmin(), train_score.min()
    gd_gamma[i_min] += 1
    print("gd: learning rate and minimal train MSE")
    print(gamma[i_min], min)

    # adding momentum
    train_score = np.zeros((len(gamma), len(delta)))
    for j in range(len(delta)):
        for i in range(len(gamma)):
            model = GradientDescend(learning_rate=gamma[i], delta_momentum=delta[j])
            scores = model.fit(X_train, z_train, X_test, z_test)

            pred_train = model.predict(X_train)
            train_score[i, j] = MSE(pred_train, z_train)
        
    i_min, min = train_score.argmin(), train_score.min()
    k, l=np.shape(train_score)
    i_min = np.unravel_index(i_min, shape=[k, l])
    gd_mom_gamma[i_min[0]] += 1
    gd_mom_delta[i_min[1]] += 1
    print("gd with momentum: learning rate, momentum and minimal train MSE")
    print(gamma[i_min[0]], delta[i_min[1]], min)


    # minibatch sgd, learning schedule
    train_score = np.zeros((len(gamma), len(delta), len(eta)))
    for j in range(len(delta)):
        for i in range(len(gamma)):
            for h in range(len(eta)):
                model = GradientDescend(optimizer="sgd", method="gd", learning_rate=gamma[i], delta_momentum=delta[j], learning_rate_decay_flag=True, learning_rate_decay=eta[h])
                scores = model.fit(X_train, z_train, X_test, z_test)

                pred_train = model.predict(X_train)
                train_score[i, j, h] = MSE(pred_train, z_train)
                

    i_min, min = train_score.argmin(), train_score.min()
    k, l, h=np.shape(train_score)
    i_min = np.unravel_index(i_min, shape=[k, l, h])
    sgd_mom_gamma[i_min[0]] += 1
    sgd_mom_delta[i_min[1]] += 1
    sgd_mom_eta[i_min[2]] += 1
    print("sgd with momentum: learning rate, momentum, learning rate decay and minimal train MSE")
    print(gamma[i_min[0]], delta[i_min[1]], eta[i_min[2]], min)


gd: learning rate and minimal train MSE
0.034 0.23300926074058112
gd with momentum: learning rate, momentum and minimal train MSE
0.08900000000000001 0.2 0.17310653933709938
sgd with momentum: learning rate, momentum, learning rate decay and minimal train MSE
0.067 0.05 0.95 0.17187237984514658
gd: learning rate and minimal train MSE
0.023000000000000003 0.17825293946556067
gd with momentum: learning rate, momentum and minimal train MSE
0.05600000000000001 0.45 0.17211254944432902
sgd with momentum: learning rate, momentum, learning rate decay and minimal train MSE
0.012 0.05 0.95 0.17187237978593364
gd: learning rate and minimal train MSE
0.045000000000000005 0.19219173388432584
gd with momentum: learning rate, momentum and minimal train MSE
0.05600000000000001 0.3 0.18233981469031074
sgd with momentum: learning rate, momentum, learning rate decay and minimal train MSE
0.023000000000000003 0.3 0.8875 0.17187238010788217
gd: learning rate and minimal train MSE
0.05600000000000001 0.173

KeyboardInterrupt: 

In [7]:
gd_gamma_opt = gamma[gd_gamma.argmax()]
gd_mom_gamma_opt = gamma[gd_mom_gamma.argmax()]
sgd_mom_gamma_opt = gamma[sgd_mom_gamma.argmax()]

gd_mom_delta_opt = delta[gd_mom_delta.argmax()]
sgd_mom_delta_opt = delta[sgd_mom_delta.argmax()]

sgd_mom_eta_opt = eta[sgd_mom_eta.argmax()]

%store gd_gamma_opt
%store gd_mom_gamma_opt
%store sgd_mom_gamma_opt
%store gd_mom_delta_opt
%store sgd_mom_delta_opt
%store sgd_mom_eta_opt


Stored 'gd_gamma_opt' (float64)
Stored 'gd_mom_gamma_opt' (float64)
Stored 'sgd_mom_gamma_opt' (float64)
Stored 'gd_mom_delta_opt' (float64)
Stored 'sgd_mom_delta_opt' (float64)
Stored 'sgd_mom_eta_opt' (float64)
